In [0]:
#@title Params
 

document_parser_lib_version='1.1.11'#@param {type:"string"}
charter_filename='/content/\u0423\u0441\u0442\u0430\u0432 - \u0413\u041F\u041D-\u0422\u0440\u0430\u043D\u0441\u043F\u043E\u0440\u0442_\u0413\u041E\u0421\u0410-2018.docx'#@param {type:"string"}
# _git_branch = 'protocols-4'
_git_branch = 'charters'#@param {type:"string"}

# INIT

## pull code

In [0]:

import json
import subprocess
import sys
import time

import numpy as np
from IPython.core.display import display, HTML
from google.colab import files

!pip install overrides

Ничто = None



def exec(x):
  r = subprocess.check_output(x, shell=True)
  r = r.decode('unicode-escape').encode('latin1').decode('utf8')
  print(r)


print(f"fetching code from GitHub.....{_git_branch}")
try:
  exec('rm -r nlp_tools')
except:
  pass
exec(f'git clone --single-branch --branch {_git_branch} https://github.com/nemoware/analyser.git nlp_tools')

print('🦊 GIT revision:')
exec('cd nlp_tools\ngit rev-list --reverse HEAD | awk "{ print NR }" | tail -n 1\ngit branch\ngit log -3 --pretty=%B')

sys.path.insert(0, 'nlp_tools')

print('❤️importing Code from GitHub ... DONE')


#----
import matplotlib as mpl
from documents import TextMap
from colab_support.renderer import HtmlRenderer
from legal_docs import DocumentJson
 

class DemoRenderer(HtmlRenderer):
  def render_color_text(self, tokens, weights, colormap='coolwarm', print_debug=False, _range=None, separator=' '):
    html = self.to_color_text(tokens, weights, colormap, print_debug, _range, separator=separator)
    display(HTML(html))

  def to_color_text(self, tokens, weights, colormap='coolwarm', print_debug=False, _range=None, separator=' '):
    return super()._to_color_text(tokens, weights, mpl, colormap=colormap, _range=_range, separator=separator)

   
renderer_ = DemoRenderer()

def print_json_summary(cd:DocumentJson):
  wordsmap = TextMap(cd.normal_text, cd.tokenization_maps['$words'])
  print(f'read file {cd.filename}')

  for tag in cd.tags:
    span = tag.span
    _map = cd.tokenization_maps[tag.span_map]
    print(tag)
 
  

In [0]:
!pip install pyjarowinkler

### Init document-parser lib

In [0]:
lib_version = '1.1.10'
import os
if not os.path.isfile(f'document-parser-{lib_version}-distribution.zip'):
  !wget https://github.com/nemoware/document-parser/releases/download/$lib_version/document-parser-$lib_version-distribution.zip
if not os.path.isdir(f'document-parser-{lib_version}'):
  !unzip document-parser-$lib_version-distribution.zip

 
os.environ ['documentparser']=f'/content/document-parser-{lib_version}'
from integration.word_document_parser import WordDocParser, join_paragraphs
wp = WordDocParser()

### imports

In [0]:
import os
import pickle
import unittest

import numpy as np

from contract_parser import ContractAnlysingContext, ContractDocument
from contract_patterns import ContractPatternFactory
from legal_docs import LegalDocument
 
from ml_tools import *

# from headers_detector import doc_features, load_model, make_headline_attention_vector
from hyperparams import HyperParameters
from protocol_parser import protocol_votes_re
 

## render tools

In [0]:
def json2html(cd:DocumentJson):
  wordsmap = TextMap(cd.normal_text, cd.tokenization_maps['words'])
  markup_vector = np.zeros(len(wordsmap))
 
  for hd in cd.headers:    
    span = hd['span']      
    markup_vector[span[0]:span[1]] += 2

  for tag in cd.attributes:
    if 'span' in  cd.attributes[tag]:
      span = cd.attributes[tag]['span']
      confidence = cd.attributes[tag]['confidence']
      markup_vector[span[0]:span[1]] += confidence
    else:
      warnings.warn(f'{tag} has no span')

  return renderer_.to_color_text(wordsmap.tokens, markup_vector)

def print_headers(d: LegalDocument):
  for p in d.paragraphs:
    print('\t --> 📂', d.substr(p.header))

## 💅 Init Embedder(s)

In [0]:
# from protocol_parser import  ProtocolPatternFactory
from tf_support.embedder_elmo import ElmoEmbedder
# from contract_patterns import ContractPatternFactory
elmo_embedder = ElmoEmbedder()
elmo_embedder_default = ElmoEmbedder(layer_name="default")

# protocols_factory = ProtocolPatternFactory(elmo_embedder)
# contracts_factory = ContractPatternFactory(elmo_embedder)

In [0]:
 
from legal_docs import tokenize_doc_into_sentences_map, ContractValue
from ml_tools import *
from parsing import ParsingContext
from patterns import *
 
from text_tools import *

# legal_docs.py
from tf_support.embedder_elmo import ElmoEmbedder
 



# C CHARTER (mock json)

In [0]:
make_mock=False

from structures import OrgStructuralLevel
from contract_parser import find_value_sign_currency_attention 

if make_mock:

  results = wp.read_doc('/content/МНГ Устав.docx')
  for d in results['documents'][:1]:  # XXX
    if 'CHARTER' == d['documentType']:    
      charter = join_paragraphs(d, 'no_id')

  charter.tags=[]


  values: [ContractValue] = find_value_sign_currency_attention(charter)
  # print(values)

  # values_list = []
  # k=0;
  # for v in values:
  #   k+=1;
  #   v.parent.kind=f'constraint_{k}'
  #   as_list = v.as_list()
  #   charter.tags+=as_list
  #   values_list+=as_list

  #   for xx in as_list:
  #     print(xx.get_key())

  tag_ceo = SemanticTag(f'competence-{OrgStructuralLevel.CEO.name}', f'{OrgStructuralLevel.CEO.name}', (7982, 8951))
  tag_all = SemanticTag(f'competence-{OrgStructuralLevel.AllMembers.name}', f'{OrgStructuralLevel.AllMembers.name}', (2512, 3168))
  
  organs = [
                tag_ceo,
                tag_all
                ]

  competences = [
                SemanticTag('finance', '', (8410, 8470), parent=tag_ceo),
                SemanticTag('deals', '', (8474, 8555), parent=tag_ceo),
                SemanticTag('court', '', (8715,8825), parent=tag_ceo)                             
                ]
  # charter.tags.append(block1)

  charter.tags += organs
  charter.tags += competences

  k=0
  for tag in competences:
    
    for contract_value in values:
      k+=1
      z ='min'
      if contract_value.sign.value>0:
        z ='max'
      contract_value.parent.kind=f'constraint-{z}'
      # for v in contract_value.as_list():
      if tag.is_nested(contract_value.parent.span):
        contract_value.parent.set_parent_tag ( tag)
        print (contract_value.parent, ' IS NESTED IN', tag.span)
        charter.tags+=contract_value.as_list()
        print(contract_value.value.get_key())
    
      
        
  for tag in charter.tags:
    print(tag.get_key())
  


  json_str=charter.to_json()

  json_obj = DocumentJson.from_json(json_str)
  with open('charter.json', 'w') as file: 
    file.write(json_str)



  # 7982

  # test_dists = relu (1-test_pattern._eval_distances(doc.embeddings), 0.5)
  # v = relu(test_dists, 0.5)
  # renderer_.render_color_text(charter.tokens, v)
  # show_json(charter)

In [0]:
# print(charter.tags)
if make_mock:
  html = json2html(json_obj)
  display(HTML(html))

In [0]:
if make_mock:
  jj = charter.to_json()
  parsed = json.loads(jj)
  del parsed['tokenization_maps']
  print(json.dumps(parsed, indent=4, sort_keys=True))

# CHARTER: parse

1. find competence-sections
2. 

In [0]:
use_old_parser=False

if use_old_parser:
  from charter_parser import CharterDocumentParser
  from charter_patterns import CharterPatternFactory


  charter_pattern_factory = CharterPatternFactory(elmo_embedder)
  CDP = CharterDocumentParser(charter_pattern_factory)


In [0]:
if use_old_parser:
  charter.embedd_tokens(elmo_embedder)

In [0]:
if use_old_parser:
  CDP.doc = charter

### competence_attention (not used??)

In [0]:
if use_old_parser:
  v = CDP._make_competence_attention_v()
  renderer_.render_color_text(charter.tokens_cc, v)

### Sections

In [0]:
if use_old_parser:
  sections = CDP.sections_finder.find_sections(charter, CDP.pattern_factory, CDP.pattern_factory.headlines, headline_patterns_prefix='headline.' )

In [0]:


if use_old_parser:
  print('sections:'.upper())
  for key in sections.keys():
    section = sections[key]
    print(section.subdoc.text)
  print()
  print('sections_filtered:'.upper())


  sections_filtered = CDP._get_head_sections()
  for key in sections_filtered.keys():
    section = sections[key]
    print(section.subdoc.text)

In [0]:
def get_tags_attention(self, tags):
  _attention = np.zeros(self.__len__())

  for t in tags:
    _attention[t.as_slice()] += t.confidence
  return _attention

def as_tag(self):
  st = SemanticTag(self.type, None, (self.subdoc.start, self.body.end))
  st.confidence = self.confidence
  return st

if use_old_parser:
  tags=[]
  for key in sections_filtered.keys():
    section = sections[key]
    tags.append( as_tag(section) )

  renderer_.render_color_text(charter.tokens, get_tags_attention(charter,tags))

In [0]:
# print_headers(charter)

### Constraints

In [0]:
if use_old_parser:
  for section_name in sections_filtered:
    section = sections_filtered[section_name].body


    all_margin_values: PatternSearchResults = section.find_sentences_by_pattern_prefix(section_name, CDP.pattern_factory,
                                                                                        'sum__')
    print(all_margin_values)
    # for c in all_margin_values.constraints:
    #   print(c)
    
    # print(section_name)
    # value_constraints, charity_constraints, all_margin_values, charity_constraints = CDP._find_constraints_in_section(
    #   org_level=section_name, section=section)

# New

## Patterns

In [0]:
competence_headline_pattern_prefix = 'headline'
comp_str_pat=[]
comp_str_pat += [ [f'{competence_headline_pattern_prefix}/{ol.name}',  ol.display_string ] for ol in OrgStructuralLevel]
comp_str_pat += [ [f'{competence_headline_pattern_prefix}/comp-q/{ol.name}', "к компетенции " + ol.display_string +' относятся следующие вопросы'] for ol in OrgStructuralLevel]
comp_str_pat += [ [f'{competence_headline_pattern_prefix}/comp/{ol.name}', "компетенции " + ol.display_string] for ol in OrgStructuralLevel]
comp_str_pat

## classes

In [0]:
from legal_docs import embedd_sentences



class CharterDocument5(LegalDocument):

  def __init__(self, doc: LegalDocument):
    super().__init__('')
    if doc is not None:
      self.__dict__ = doc.__dict__

    self.sentence_map: TextMap = None
    self.sentences_embeddings = None

    self.distances_per_sentence_pattern_dict = {}

    self.charity_tags=[]
    # self.agents_tags: [SemanticTag] = []
    # self.org_level: [SemanticTag] = []
    # self.agenda_questions: [SemanticTag] = []
    self.margin_values: [ContractValue] = []

  def subdoc_slice(self, __s: slice, name='undef'):
    sub = super().subdoc_slice(__s, name)
    span = [max((0, __s.start)), max((0, __s.stop))]

    sentences_span = self.tokens_map.remap_span(span, self.sentence_map)
    _slice = slice(sentences_span[0], sentences_span[1])
    sub.sentence_map = self.sentence_map.slice(_slice)
  
    if self.sentences_embeddings is not None:
      sub.sentences_embeddings = self.sentences_embeddings[_slice]

    return sub

    
  def get_tags(self) -> [SemanticTag]:
    tags = []
    tags += self.charity_tags
    # tags += self.org_level
    # tags += self.agenda_questions
    for mv in self.margin_values:
      tags += mv.as_list()

    return tags

   

class CharterParser2(ParsingContext):

  patterns_dict = [
    # ['headline.head.directors.1', '0. Компетенции Совета директоров Общества'],    
    # ['headline.head.directors.2', 'К компетенции Совета директоров относятся следующие вопросы'],    

    # ['headline.head.all.n.a', 'Компетенции Общего собрания акционеров Общества']
    
  ] + comp_str_pat

  def __init__(self, embedder, elmo_embedder_default: ElmoEmbedder):
    ParsingContext.__init__(self, embedder)
    self.elmo_embedder_default = elmo_embedder_default

    patterns_te = [p[1] for p in CharterParser2.patterns_dict]
    self.patterns_embeddings = elmo_embedder_default.embedd_strings(patterns_te)
  
  def ebmedd(self, doc: CharterDocument5):
    doc.sentence_map = tokenize_doc_into_sentences_map(doc, 200)

    ### ⚙️🔮 SENTENCES embedding
    doc.sentences_embeddings = embedd_sentences(doc.sentence_map, self.elmo_embedder_default)

    doc.distances_per_sentence_pattern_dict = calc_distances_per_pattern_dict(doc.sentences_embeddings,
                                                                              self.patterns_dict,
                                                                              self.patterns_embeddings)



results = wp.read_doc(charter_filename)
# results = wp.read_doc('/content/ЕЮ Устав.docx')

for d in results['documents'][:1]:  # XXX
  if 'CHARTER' == d['documentType']:    
    charter = join_paragraphs(d, 'no_id')
    charter = CharterDocument5(charter)

print_headers(charter)

In [0]:
charter_parser = CharterParser2(elmo_embedder, elmo_embedder_default)
charter_parser.ebmedd(charter)

## Find org level sections
 - find all headers spans
 - compare headers with patterns 
 - find interresting spans

In [0]:
header_slices = [p.header.as_slice() for p in charter.paragraphs]
header_slices

remap to sentences

In [0]:
header_slices_sent = charter.tokens_map.remap_slices(header_slices, charter.sentence_map)
header_slices_sent


In [0]:
section_ends = [ x[0] for x in header_slices_sent ] + [ len(charter.sentence_map) ] #+end of doc
section_ends

Available sentence pattern (names)

In [0]:
charter.distances_per_sentence_pattern_dict.keys()

In [0]:
# fi 
# av_competence_headline_pattern = max_exclusive_pattern_by_prefix(charter.distances_per_sentence_pattern_dict, competence_headline_pattern_prefix)
# av_competence_headline_pattern = relu(av_competence_headline_pattern, 0.66)

# av_interresting_headers = headers_att * av_competence_headline_pattern
# #words map
# # sections_spans_words = charter.sentence_map.remap_slices(v, charter.tokens_map)
 
# # v = charter.distances_per_sentence_pattern_dict['headline.head.directors.n']# - 0.1*charter.distances_per_sentence_pattern_dict['headline.head.directors.not'] 

# renderer_.render_color_text(charter.sentence_map.tokens, av_interresting_headers, _range=(0,1), separator='¶<br>')
# # renderer_.render_color_text(charter.tokens_cc, sections_spans_words, _range=(0,1) )

In [0]:
charter_parser.patterns_dict

## Find relevant (org struct) paragraphs

In [0]:
from doc_structure import get_tokenized_line_number
from ml_tools import calc_distances_per_pattern_dict

# charter.
def headers_as_sentences(doc: LegalDocument, normal_case=True, strip_number=True):
  _map = doc.tokens_map
  if normal_case:
    _map = doc.tokens_map_norm

  numbered = [_map.slice(p.header.as_slice()) for p in doc.paragraphs]
  stripped = []

  for s in numbered:
    if strip_number:
      a = get_tokenized_line_number(s.tokens, 0)
      _, span, _, _ = a
      line = s.text_range([span[1], None]).strip()
    else:
      line = s.text

    # print('😱', line)
    stripped.append(line)

  return stripped


def map_headlines_to_patterns(charter, charter_parser, pattern_prefix:str, pattern_suffixes:[str]):
  """
  1. Collect headers as string, strip numbers, normalize case
  2. Embedd heades with sentence embedder
  3. Map headers to patterns
  """
  headers = headers_as_sentences(charter)
  headers_embedding = charter_parser.elmo_embedder_default.embedd_strings(headers)
  
  # #debug
  # for e in headers_embedding:
  #   print(e[0:3])
  
  header_to_pattern_distances = calc_distances_per_pattern_dict(headers_embedding,
                                                                charter_parser.patterns_dict,
                                                                charter_parser.patterns_embeddings)
  
  patterns_by_headers = [()]*len(headers)
  for e in range(len(headers)):
    # for each header 
    max_confidence = 0
    for pattern_suffix in pattern_suffixes:
      pattern_name = pattern_prefix+pattern_suffix
      # find best pattern
      confidence = header_to_pattern_distances[pattern_name][e]
      if confidence > max_confidence and confidence > 0.7:
        patterns_by_headers[e] = (pattern_name, pattern_suffix, confidence, headers[e], charter.paragraphs[e])
        max_confidence = confidence
  
  return patterns_by_headers, header_to_pattern_distances



#------------------------DEBUG CUT

p_suffices = [ol.name for ol in OrgStructuralLevel] 
p_suffices += ['comp/'+ol.name for ol in OrgStructuralLevel] 
 
patterns_by_headers, header_to_pattern_distances = map_headlines_to_patterns(charter, charter_parser, competence_headline_pattern_prefix+'/', p_suffices )
for k in patterns_by_headers :
  if k:
    print(k)


print('\n\nheadline/comp/AllMembers')
renderer_.render_color_text(headers_as_sentences(charter), header_to_pattern_distances['headline/comp/AllMembers'], _range=(0,1), separator='😱 <br>')


In [0]:
def get_values_attention(subdoc, values:List[ContractValue]):
  numbers_attention = np.zeros(len(subdoc.tokens_map))
    
  for v in values:
    numbers_attention  [ v.value.as_slice() ] += v.value.confidence
    numbers_attention  [ v.currency.as_slice() ] += v.currency.confidence
    numbers_attention  [ v.sign.as_slice() ] += v.sign.confidence
    numbers_attention  [ v.parent.as_slice() ] += v.parent.confidence

  return numbers_attention/2

for p_mapping in patterns_by_headers:
  if p_mapping:
    paragraph =  p_mapping[4]
    org_level =  p_mapping[1]

    
    subdoc = charter.subdoc_slice(paragraph.body.as_slice())
    values:List[ContractValue] = find_value_sign_currency_attention(subdoc, None)

    #color 
    numbers_attention = get_values_attention(subdoc, values)
    
    # for v in values:
    #   numbers_attention  [ v.value.as_slice() ] = 1
    #   numbers_attention  [ v.currency.as_slice() ] = 1
    #   numbers_attention  [ v.sign.as_slice() ] = 1

    print('%'*100)
    print(org_level)
    # renderer_.render_color_text(subdoc.tokens_map.tokens, numbers_attention, _range=(0,1))

In [0]:
# class CharterPatternFactory(AbstractPatternFactoryLowCase):

#   def __init__(self, embedder):
#     AbstractPatternFactoryLowCase.__init__(self, embedder)
#     # _build_order_patterns(self) 

#     # self.embedd()

# def _build_order_patterns():
  
#   def cp(r, name, tuples):
#     r +=[  ['order/'+name, ' '.join(tuples)] ]

#   ret = []
#   # cp(ret, 'consent', ('Порядок', 'одобрения сделок', 'в совершении которых имеется заинтересованность'))
#   cp(ret, 'solution', ('', 'принятие решений', 'о совершении сделок'))
#   cp(ret, 'consent_1', ('', 'одобрение заключения', 'изменения или расторжения какой-либо сделки Общества'))

#   prefix = 'принятие решения о согласии на совершение или о последующем одобрении'

#   cp(ret, 'deal.1', (prefix, 'cделки', ', стоимость которой равна или превышает'))
#   cp(ret, 'deal.2', (prefix, 'cделки', ', стоимость которой составляет менее'))

#   return ret


# order_patterns = _build_order_patterns() 
# order_patterns
 
 
 

In [0]:
from enum import Enum, unique, EnumMeta
from typing import List
from structures import DisplayStringEnumMeta

@unique
class CharterSubject(Enum, metaclass=DisplayStringEnumMeta):
   
  Deal = 0, 'Сделка'
  Charity = 1, 'Благотворительность'
  Other = 2, 'Другое'
  Lawsuit = 3, 'Судебные издержки'
  RealEstate = 4, 'Недвижимость'
  Insurance = 5, 'Страхование'
  Consulting = 6, 'Консультационные услуги'


 
strs_subjects_patterns = {
    
    # CharterSubject.Charity:[
    #   'одобрение внесения Обществом пожертвований на политические или благотворительные цели',  
    #   'предоставление безвозмездной финансовой помощи',
    #   'согласование сделок дарения',
    #   'одобрение договоров спонсорского и благотворительного характера',
    #   'передача в безвозмездное пользование',
    #   'совершение сделок мены, дарения, предусматривающих безвозмездное отчуждение '
    # ],

     
    CharterSubject.Deal:[
      'принятие решений о совершении сделок'
    ],

    CharterSubject.Charity:[
      'пожертвований на политические или благотворительные цели',  
      'предоставление безвозмездной финансовой помощи',
      'сделок дарения',
      'договоров спонсорского и благотворительного характера',
      'передача в безвозмездное пользование',
      'мены, дарения, безвозмездное отчуждение '
    ],

    CharterSubject.Lawsuit:[
      'о начале/урегулировании любых судебных споров и разбирательств',
      'заключении Обществом мирового соглашения по судебному делу с ценой иска '    
    ],

    CharterSubject.RealEstate:[
      'стоимость отчуждаемого имущества',
      'сделки с имуществом Общества'
    ],

    # CharterSubject.Insurance:[
    #   'одобрение заключения договоров страхования',
    #   'одобрение изменения, продления, возобновления или расторжения Обществом договоров страхования'
    #   'принятие решений о совершении сделок страхования'
    # ],

    CharterSubject.Insurance:[
      'заключение договоров страхования',
      'возобновления договоров страхования'
      'совершение сделок страхования'
    ],

    

    CharterSubject.Consulting:[
      'договора оказания консультационных услуг',
      'заключения агентского договора',
      'согласование договора оказания консультационных услуг или агентского договора',
      'оказания обществу информационных юридических услуг '
    ],

    CharterSubject.Other:[
      'решения о взыскании с Генерального директора убытков',      
      'заключение договоров об отступном , новации и/или прощении долга , договоров об уступке права требования и переводе долга',
      'о выдаче или получении Обществом векселей , производстве по ним передаточных надписей , авалей , платежей',
      'нецелевое расходование Обществом денежных средств'
    ]

    
    
    

    # CharterSubject.Consulting:[
    #   'одобрение договора оказания консультационных услуг',
    #   'одобрение заключения агентского договора',
    #   'согласование заключения договора оказания консультационных услуг или агентского договора'
    # ]
}
 


str_consent_patterns=[
  'принятие решений о совершении сделок',
  'принятие решения о согласии на совершение или о последующем одобрении сделок',
  'одобрение заключения изменения или расторжения какой-либо сделки Общества',
  'принятие решения о согласии на совершение или о последующем одобрении cделки стоимость которой равна или превышает'
]

def build_sentence_patterns(strings, prefix, embedder):
  ret = []
  for txt in strings:
    ret.append ( [f'{prefix}/{len(ret)}',  txt ]  )

  emb =  embedder.embedd_strings(strings)
  return ret, emb

emb_subj_patterns={}
for subj in strs_subjects_patterns.keys():
  strings = strs_subjects_patterns[subj]
  patterns, patterns_emb = build_sentence_patterns(strings, f'subject/{subj.name}', elmo_embedder_default)

  emb_subj_patterns[subj] = {
    'patterns':patterns, 
    'embedding':patterns_emb    
  }
  

consent_patterns, consent_patterns_emb = build_sentence_patterns(str_consent_patterns, 'consent', elmo_embedder_default)

###-----debug-----
emb_subj_patterns[CharterSubject.Insurance]['patterns']

In [0]:
# order_patterns_emb = elmo_embedder_default.embedd_strings([p[1] for p in order_patterns])


In [0]:
from contract_parser import _find_most_relevant_paragraph
from legal_docs import remap_attention_vector


def find_charity_paragraphs(org_level, subdoc, subject_attention ):
  paragraph_span, confidence, paragraph_attention_vector = _find_most_relevant_paragraph(subdoc,
                                                                                             subject_attention,
                                                                                             min_len=20,
                                                                                             return_delimiters=False)
    
  if confidence>0.6:
    subject_tag = SemanticTag(f'{org_level}_{CharterSubject.Charity.name}', CharterSubject.Charity.name, paragraph_span)
    subject_tag.offset(subdoc.start)


    print(org_level)
    print(confidence, subdoc.tokens_map.text_range(paragraph_span))
    

    return subject_tag




from contract_parser import _find_most_relevant_paragraph, find_value_sign_currency_attention
from legal_docs import remap_attention_vector, ContractValue, LegalDocumentExt
from ml_tools import *




def find_charity_paragraphs(org_level:SemanticTag, subdoc, subject_attention):
  paragraph_span, confidence, paragraph_attention_vector = _find_most_relevant_paragraph(subdoc,
                                                                                         subject_attention,
                                                                                         min_len=20,
                                                                                         return_delimiters=False)

  if confidence > 0.6:
    subject_tag = SemanticTag(f'{CharterSubject.Charity.name}', CharterSubject.Charity.name, paragraph_span, parent=org_level)
    subject_tag.offset(subdoc.start)

    print(org_level)
    print(confidence, subdoc.tokens_map.text_range(paragraph_span))

    return subject_tag






def get_charter_subj_attentions(subdoc: LegalDocumentExt, emb_subj_patterns):
  _distances_per_subj = {}
  for subj in emb_subj_patterns.keys():
    patterns = emb_subj_patterns[subj]['patterns']
    embedding = emb_subj_patterns[subj]['embedding']
    patterns_distances = calc_distances_per_pattern_dict(subdoc.sentences_embeddings,
                                                         patterns,
                                                         embedding)

    subj_av = relu(max_exclusive_pattern_by_prefix(patterns_distances, f'subject/{subj.name}/'), 0.6)
    subj_av_words = remap_attention_vector(subj_av, subdoc.sentence_map, subdoc.tokens_map)

    _distances_per_subj[subj] = {
      'words': subj_av_words,
      'sentences': subj_av,
    }
  return _distances_per_subj




def charter_subjects_attribution(subdoc: LegalDocumentExt, emb_subj_patterns, parent_org_level_tag:SemanticTag):
  """
  :param subdoc:
  :param emb_subj_patterns:

        emb_subj_patterns[subj] = {
          'patterns':patterns,
          'embedding':patterns_emb
        }

  :return:
  """

  # ---
  subject_attentions_map = get_charter_subj_attentions(subdoc, emb_subj_patterns)
  # ---

  # collect sentences having constraint values
  sentence_spans = []
  for value in values:
    sentence_span = subdoc.tokens_map.sentence_at_index(value.parent.span[0], return_delimiters=True)
    if sentence_span not in sentence_spans:
      sentence_spans.append(sentence_span)
      sentence_spans = merge_colliding_spans(sentence_spans, eps=1)



  # ---
  # attribute sentences to subject

  constraint_tags = []
  for span in sentence_spans:
    print()
    print('- ' * 30)
    print('VALUE', span, subdoc.tokens_map.text_range(span))

    max_confidence = 0
    best_subject = CharterSubject.Other

    for subj in subject_attentions_map.keys():
      av = subject_attentions_map[subj]['words']

      confidence_region = av[span[0]:span[1]]
      confidence = estimate_confidence_by_mean_top_non_zeros(confidence_region)

      # renderer_.render_color_text(subdoc.tokens_map[span[0]:span[1]], confidence_region, _range=(0, 1))
      print(subj, confidence)

      if confidence> max_confidence:
        max_confidence = confidence
        best_subject = subj

    constraint_tag = SemanticTag( f'{best_subject.name}', best_subject, span, parent=parent_org_level_tag )
    constraint_tags.append(constraint_tag)
    # nest values
    for value in values:
      if constraint_tag.is_nested(value.parent):
        value.parent.set_parent_tag(constraint_tag)
        
    print('~ ' * 30)

  return constraint_tags, values



for p_mapping in patterns_by_headers:
  if p_mapping:
    paragraph =  p_mapping[4]
    org_level =  p_mapping[1]

    
    subdoc = charter.subdoc_slice( paragraph.body.as_slice())
    parent_org_level_tag = SemanticTag(org_level, org_level, paragraph.body.span)
    
    consent_patterns_distances = calc_distances_per_pattern_dict(subdoc.sentences_embeddings,
                                                                consent_patterns,
                                                                consent_patterns_emb)
    
    # charity_patterns_distances = calc_distances_per_pattern_dict(subdoc.sentences_embeddings,
    #                                                             consulting_patterns,
    #                                                             consulting_patterns_emb)
    
    # print(order_patterns_distances.keys())
    values:List[ContractValue] = find_value_sign_currency_attention(subdoc, None)
    numbers_attention = get_values_attention(subdoc, values)

    consent_v = relu(max_exclusive_pattern_by_prefix(consent_patterns_distances, 'consent/'),0.5)
    consent_words = remap_attention_vector(consent_v, subdoc.sentence_map, subdoc.tokens_map)

    # charity_subj_av = relu(max_exclusive_pattern_by_prefix(charity_patterns_distances, 'consulting/'),0.6)
    # charity_subj_av_words = remap_attention_vector(charity_subj_av, subdoc.sentence_map, subdoc.tokens_map)
    # order_patterns_distances['order/consent']
    # renderer_.render_color_text(subdoc.sentence_map.tokens, v, _range=(0,1), separator='😱 <br>')
    
    # charity_patterns_distances_words = remap_attention_vector(v, subdoc.sentence_map, subdoc.tokens_map)

    
    
    # dd = get_charter_subj_attentions(subdoc, emb_subj_patterns)
    # charity_subj_av_words = dd[CharterSubject.Charity]['words']
    subject_attentions_map = get_charter_subj_attentions(subdoc, emb_subj_patterns)

    charity_subj_av_words = subject_attentions_map[CharterSubject.Charity]['words']
    charity_tag = find_charity_paragraphs(parent_org_level_tag, subdoc, (charity_subj_av_words+consent_words)/2)
    print(charity_tag)
    if charity_tag is not None:
      charter.charity_tags.append(charity_tag)

 


    # renderer_.render_color_text(subdoc.tokens_cc, charity_subj_av_words + numbers_attention , _range=(0,2) )

In [0]:


for p_mapping in patterns_by_headers:
  if p_mapping:
    paragraph =  p_mapping[4]
    org_level =  p_mapping[1]

    
    subdoc = charter.subdoc_slice( paragraph.body.as_slice())
    
    consent_patterns_distances = calc_distances_per_pattern_dict(subdoc.sentences_embeddings,
                                                                consent_patterns,
                                                                consent_patterns_emb)
    
     
    

    consent_v = relu(max_exclusive_pattern_by_prefix(consent_patterns_distances, 'consent/'),0.5)
    consent_words = remap_attention_vector(consent_v, subdoc.sentence_map, subdoc.tokens_map)
     
     
    values:List[ContractValue] = find_value_sign_currency_attention(subdoc, None)

    #collect sentences with constraint values
    sentence_spans=[]
    for value in values:
      sentence_span = subdoc.tokens_map.sentence_at_index(value.parent.span[0], return_delimiters=True)      
      if sentence_span not in sentence_spans:
        sentence_spans.append(sentence_span) 
    # sentence_spans = merge_colliding_spans(sentence_spans, eps=1)
     


    subject_attentions_map = get_charter_subj_attentions(subdoc, emb_subj_patterns)

    # attribute sentences to subject
    
    for span in sentence_spans:
      print ( )
      print ('-'*30)
      print ('VALUE', sentence_span, subdoc.tokens_map.text_range(span))
      for subj in subject_attentions_map.keys():
        av = subject_attentions_map[subj]['words']
        
        confidence_region = av[span[0]:span[1]]
        confidence = estimate_confidence_by_mean_top_non_zeros(confidence_region)

        renderer_.render_color_text(subdoc.tokens_map[span[0]:span[1]], confidence_region , _range=(0,1) )
        print(subj,  confidence )
      print ('~'*30)
    numbers_attention = get_values_attention(subdoc, values)


    

    charity_subj_av_words = subject_attentions_map[CharterSubject.Consulting]['words']
    charity_tag = find_charity_paragraphs(org_level, subdoc, numbers_attention*(charity_subj_av_words+consent_words)/2)
    print(charity_tag)
    
    



## Contraint values

In [0]:
values:List[ContractValue] = find_value_sign_currency_attention(charter, None)

#color 
numbers_attention = np.zeros(len(charter.tokens_map))
numbers_confidence = np.zeros(len(charter.tokens_map))
for v in values:
  numbers_confidence [ v.value.as_slice() ] +=v.value.confidence
  numbers_attention  [ v.value.as_slice() ] = 1
  numbers_attention  [ v.currency.as_slice() ] = 1
  numbers_attention  [ v.sign.as_slice() ] = 1

renderer_.render_color_text(charter.tokens_map.tokens, numbers_attention, _range=(0,1))


In [0]:
for v in values:
  v
  print(v.value)

In [0]:
json_str=charter.to_json()

json_obj = DocumentJson.from_json(json_str)
with open('charter.json', 'w') as file: 
  file.write(json_str)